5주차 예습과제에서는 6주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [1]:
# GPU 켜져 있는지 확인
!nvidia-smi

Tue Aug  6 11:42:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# peft

peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [4]:
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
# base configuration class to store the configuration of a PeftModel
config = PeftConfig.from_pretrained(peft_model_id)

''' 양자화를 위한 모델 구성 '''
bnb_config = BitsAndBytesConfig(
    # 4비트 양자화 사용
    # 모델의 매개 변수를 4비트 정밀도 형식으로 불러온다.
    # 메모리 사용량을 줄일 수 있다.
    load_in_4bit = True,
    # 4비트 값에 대해 이중 양자화를 사용
    # 이중 양자화는 양자화된 값의 정확도를 향상시켜 모델의 성능을 개선할 수 있다.
    bnb_4bit_use_double_quant = True,
    # 4비트 표현을 위한 양자화 유형을 지정
    # nf4: 4-bit NormalFloat Quantization
    bnb_4bit_quant_type = "nf4",
    # 계산을 위한 데이터 유형을 설정
    # torch.bfloat16: 16비트 부동 소수점 형식. > 정밀도와 메모리 사용 사이의 균형을 제공
    bnb_4bit_compute_dtype = torch.bfloat16
)

# 사전 학습된 언어 모델 로드
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             quantization_config = bnb_config,
                                             # 전체 모델을 GPU 0에 로드
                                             device_map={"":0})
# PEFT 모델 로드
# 이때, PEFT 모델의 추가 매개변수를 사전 학습된 언어 모델에 통합한다
model = PeftModel.from_pretrained(model, peft_model_id)
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

...

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXSdpaAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
            

# next token generation

## temperature
- temperature scaling이란 모델의 다음 토큰 확률분포에 변형을 가해 문장을 다양하게 생성하는 기법이다. 확률분포를 변형한다는 의미는, 대소 관계의 역전 없이 분포의 모양만을 바꾼다는 것을 말한다.
- temperature 값이 0에 가까울수록, 확률분포 모양이 원래보다 뾰족해진다. 즉, 원래 컸던 확률은 더 커지고, 작았던 확률은 더 작아진다. > 확률값이 가장 높은 토큰이 다음 토큰으로 뽑힐 가능성이 높아진다.
- temperature 값이 1이면, 확률분포에 어떤 변형도 안 가하고 그대로 사용한다.
- temperature 값이 1보다 크면, 확률분포가 평평해진다. 즉, 원래 컸던 확률과 작았던 확률 사이의 차이가 줄어든다. > 확률값이 작아서 선택되기 어려웠던 토큰들이 다음 토큰으로 선택될 가능성이 높아진다. > 다양한 문장이 생성될 수는 있으나, 생성 문장의 품질이 나빠질 수 있다.
- 참고 : https://ratsgo.github.io/nlpbook/docs/generation/inference1/
- 참고 : https://littlefoxdiary.tistory.com/46


In [9]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
    # prompt 토큰화 > PyTorch 텐서 형태로 변환 > 텐서를 GPU로 이동
    # non_blocking=True: 데이터 전송과 계산을 병렬로 수행
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    # 텍스트 생성
    gen_tokens = model.generate(tokens,  # 모델의 입력
                                # 다음 토큰의 확률분포에 따라 단어를 샘플링
                                # 모델이 만들어낸 확률은 smooth해서 낮은 확률의 토큰이 지나치게 잘 샘플링될 수 있다
                                # 이를 해결하기 위해 temperature scaling 사용.
                                do_sample = True,
                                # 확률값이 높은 토큰이 다음 토큰으로 뽑힐 가능성이 더 높아진다.
                                temperature = 0.8,
                                # 생성될 텍스트의 최대 길이는 80 토큰
                                max_length = 80)
    # 생성된 토큰을 텍스트로 변환
    # batch_decode: 여러 시퀀스를 동시에 디코딩할 때 사용. 하지만 여기에서는 시퀀스가 한 개임.
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 수 있을 것이다.​"인공지능이 인간처럼 행동할 수 있는 단계에 도달하는 것"이 지능의 핵심이다.​​인공지능이란 인간의 지능에 도전하는 것이다.​​


# QA

In [10]:
def gen(x):
    # 질문과 답변의 prompt
    q = f"### 질문: {x}\n\n### 답변:"
    print(q)

    gened = model.generate(
        # prompt를 토큰화 > PyTorch 텐서로 변환 > 텐서를 GPU로 이동
        **tokenizer(
            q,
            return_tensors = 'pt',
            # 문장 구분해주는 token_type_ids 출력 안 함
            return_token_type_ids = False
        ).to('cuda'),
        # 최대 100개의 새 토큰 생성
        max_new_tokens = 100,
        # 생성이 완료되면 조기에 중지
        early_stopping = True,
        # 다음 토큰의 확률분포에 따라 단어를 샘플링
        do_sample = True,
        # 문장의 끝을 나타내는 토큰 id 지정. 이 토큰이 나타나면 텍스트 생성을 중지.
        eos_token_id = 2,
    )
    # 생성된 토큰을 텍스트로 변환
    print(tokenizer.decode(gened[0]))

In [14]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변:
### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: (1) 운동하기 (2) 건강한 식품 먹기 (3) 충분한 수면 취하기<|endoftext|>


In [18]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변:
### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 첫 째로, 몸에 해로운 음식을 피하는 것입니다. 담배나 알콜 음료 그리고 마약을 피하면서, 두 번째로, 물을 많이 마시고 또 물이 많이 들어있는 과일과 야채를 많이 섭취하


top_k와 같은 parameter를 조절하면서 실험해보세요!

## top-k
- top-k 샘플링은 모델이 예측한 다음 토큰의 확률 분포에서 확률값이 가장 높은 k개 토큰 가운데 하나를 다음 토큰으로 선택하는 기법이다.
- k는 1 이상의 정수를 입력해야 한다.
- k=1이면 do_sample을 True로 설정하더라도, greedy search와 동일한 효과를 낸다.

In [19]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample = True,
                                temperature = 0.8,
                                max_length = 64,
                                # 다음 토큰의 확률 분포에서 확률값이 가장 높은 50개 토큰 가운데 하나를 선택
                                top_k = 50)
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결한다. 영화는 이렇게 인간과 교감하는 '로봇'의 모습을 통해 앞으로의 미래사회에서 로봇은 인간의 '가족'의 형태로 존재하게 되리라 이야기


### top_k=1 (greedy search)

In [22]:
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample = True,
                                temperature = 0.8,
                                max_length = 64,
                                # greedy search
                                top_k = 1)
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 수 있을 것으로 기대하고 있다.​​​​​​​​​​​​​​


=> 여러 번 실행해도, 동일한 텍스트를 생성한다.

### top_k = 500

In [24]:
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample = True,
                                temperature = 0.8,
                                max_length = 64,
                                top_k = 500)
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결하는 지능형 로봇들이 등장할 것이다. ​​​​​​​​​​​​​<|endoftext|>


=> 500으로 높였는데도, 말이 되는 텍스트가 나타나고 있다.

### temperature=0.1

In [26]:
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample = True,
                                temperature = 0.1,
                                max_length = 64,
                                top_k = 50)
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 수 있을 것으로 기대하고 있다.​​​​​​​​​​​​​​


### temperatue=100.0

In [34]:
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample = True,
                                temperature = 100.0,
                                max_length = 64,
                                top_k = 50)
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 신비.. 인류 지능에서.. 과학 문명적 사실 중 놀라운 몇 대목 등입니다<|endoftext|>


=> 생성 성능이 나빠졌다.